# Demo of the combination and simplification algorithm

In [7]:
import pandas as pd


## For using the library, just import the _RuleCOSIClassifier_ class from **rulecosi** package

The algorithm works with several type of tree ensembles and it uses the **sklearn** implementations.
- Bagging Trees
- RandomForests
- Gradient Boosting Trees (original implementation)
- XGBoost
- Light GBM
- CatBoost

In [8]:
#from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

#from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier

import numpy as np
from sklearn.utils import Bunch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer

### Load a sample dataset and split the data

We use the Wisconsin diagnostic breast cancer dataset. There are two classes, malignant (0) and benign (1).

In [9]:

from notebooks.IPRules.read_datasets import read_dataset
test_size = 0.2
name = "credit"

path = f'../../..'
dataset_path_name = f'{path}/data/{name}.csv'
X, y, dataset, target_value_name, pandas_dataset = read_dataset(name, dataset_path_name)

pandas_dataset.head()

,LIMIT_BAL_L_High,LIMIT_BAL_L_Low,LIMIT_BAL_L_Medium,LIMIT_BAL_L_VeryHigh,LIMIT_BAL_L_VeryLow,LIMIT_BAL_R_Low,LIMIT_BAL_R_Medium,LIMIT_BAL_R_VeryHigh,LIMIT_BAL_R_VeryLow,SEX_1,...,PAY_AMT6_L_Low,PAY_AMT6_L_Medium,PAY_AMT6_L_VeryHigh,PAY_AMT6_L_VeryLow,PAY_AMT6_R_High,PAY_AMT6_R_Low,PAY_AMT6_R_Medium,PAY_AMT6_R_VeryHigh,PAY_AMT6_R_VeryLow,defaultpaymentnextmonth
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
2,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False


In [10]:
#Define dataset
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=1)
encoded_train_pandas_dataset = pd.DataFrame(data= np.c_[X_train, y_train], columns= list(dataset['feature_names']) + [target_value_name])
encoded_test_pandas_dataset = pd.DataFrame(data= np.c_[X_test, y_test], columns= list(dataset['feature_names']) + [target_value_name])
print('Sizes (without target):')
print(f'Original size {dataset.data.shape}')
print(f'Train size {X_train.shape}')
print(f'Test size {X_test.shape}')
print(f'encoded_train_pandas_dataset size {encoded_train_pandas_dataset.shape}')
print(f'encoded_test_pandas_dataset size {encoded_test_pandas_dataset.shape}')

Sizes (without target):
Original size (30000, 200)
Train size (24000, 200)
Test size (6000, 200)
encoded_train_pandas_dataset size (24000, 201)
encoded_test_pandas_dataset size (6000, 201)


### Simplifying an XGBoost classifier

We create a XGBClassifier instance. The ensemble can be fitted, or it can be just instantiated and RuleCOSI will fit the ensemble first and then simplify it.

In [11]:
ens = XGBClassifier(random_state=1212)

This is done by instanciating a **RuleCOSIClassifier** class with the desired parameters, _n\_estimator_, _tree\_max\_depth_, _conf\_threshold_ and _min\_samples_.

In [12]:


from rulecosi import RuleCOSIClassifier

rc = RuleCOSIClassifier(base_ensemble=ens,
                        metric='f1',n_estimators=100, tree_max_depth=3, 
                        conf_threshold=0.9, cov_threshold=0.0,
                        random_state=1212, column_names=dataset.feature_names)

In [ ]:
%%time
rc.fit(X_train, y_train)

## Examining the simplified rules

The rules will be stored in the _simplified\_ruleset_ \_ attribute of the RuleCOSI object. The function _print\_rules_ print the rules and its heuristics on the console. It can also return a string object or a pandas DataFrame object to be used for further analysis. Additionally, the decimal digits displayed on the heuristics values and the condition thresholds can be modified with the _heuristics\_digits_ and the _condition\_digits_ parameters.

In [ ]:
rc.simplified_ruleset_.print_rules(heuristics_digits=4, condition_digits=1)

In [ ]:
rc.simplified_ruleset_.print_rules(return_object='dataframe',heuristics_digits=4, condition_digits=1)

In [ ]:
print(len(rc.simplified_ruleset_))

## Checking the classification performance of the simplified rule-based classifier

In [ ]:
# this function is used for counting the number of rules extracted from the tree ensemble (original ruelesets)
def get_n_rules(rulesets):
    n_rules = 0
    for ruleset in rulesets:
        for rule in ruleset:
            n_rules += 1
    return n_rules

In [ ]:
print(f'== Original XGBoost ensemble ==')
print(f'Number of trees: {rc.base_ensemble_.n_estimators} trees')
print(f'Number of rules: {get_n_rules(rc.original_rulesets_)} rules\n')

print(f'== Simplified rules ==')
rc.simplified_ruleset_.print_rules()
y_pred = rc.predict(X_test)
if isinstance(rc.base_ensemble, XGBClassifier):
    y_pred_ens = rc.base_ensemble_.predict(X_test, validate_features=False)
else:
    y_pred_ens = rc.base_ensemble_.predict(X_test)
print("Combinations: {}".format(rc.n_combinations_))
print("Time: {}\n".format(rc.combination_time_))
print(f'====== Classification performance of XGBoost ======')
print(classification_report(y_test, y_pred_ens,digits=4))
print(f'\n====== Classification performance of simplified rules ======')
print(classification_report(y_test, y_pred,digits=4))
print('\n')
